In [1]:
using Pkg
Pkg.activate("/mnt/dv/wid/projects4/SolisLemus-network-merging/")
Pkg.instantiate()

using InPhyNet, PhyloNetworks
include("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/simulation-scripts/helpers/helpers.jl")
cd("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences")

  Activating project at `/mnt/dv/wid/projects4/SolisLemus-network-merging`


In [37]:
# "Instance" variables
ntaxa = 500
replicatenum = 1
ngt = 1000
seq_len = 500
ils_level = "low"
m = 25
dmethod = "AGIC"

true_net = load_true_net_ils_adjusted_level1(ntaxa, replicatenum, ils_level)
# for e in true_net.edge
#     if e.length == -1. e.length = 0.473 end
# end

HybridNetwork, Rooted Network
1048 edges
1033 nodes: 501 tips, 16 hybrid nodes, 516 internal tree nodes.
tip labels: t3, t4, t22, t23, ...
(((((((t3:5.453,t4:5.453):5.453,(((((t22:0.676,t23:0.676):0.545,t18:1.174):4.021,t5:5.195):5.453,(((t13:1.469,(t17:1.307,(t19:1.128,t20:1.128):0.545):0.545):3.407,(t12:1.503)#H1:3.373::0.819):3.565,(((t8:2.227,t9:2.227):4.402,(t10:1.558,t11:1.558):5.071):0.545,(t6:2.67,(t16:1.4,((t24:0.545,t25:0.545):0.702,t21:1.018):0.545):1.27):4.205):1.567):4.207):1.582,((t2:5.453,(((t14:1.433,t15:1.433):0.545,#H1:0.545::0.181):1.128,t7:2.631):5.453):2.256,t1:5.453):3.878):1.308):5.453,(((((((t233:1.863,(t235:1.279,((t241:0.733,t242:0.733):0.545)#H9:0.543::0.974):0.584):0.545,(t243:0.545,t244:0.545):1.909):1.085,t229:3.489):3.454,(t227:5.047,(t228:4.63,(t230:2.804,t231:2.804):1.826):0.545):1.896):0.545,(((t236:0.966,t237:0.966):4.245)#H10:0.61::0.997,((((((((t238:0.802)#H11:0.0::0.905,t239:0.802):1.379,((t249:0.545,t250:0.545):1.653,t234:1.835):0.545):0.545,(t245

In [38]:
# File paths
data_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/temp_data/"
checkpoint_dir = joinpath(pwd(), "temp_data")
if !isdir(checkpoint_dir) mkdir(checkpoint_dir) end

truegt_file = joinpath(checkpoint_dir, "truegt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(ils_level).treefile")
seq_file_prefix = joinpath(checkpoint_dir, "seqfile_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).phy")
estgt_file = joinpath(checkpoint_dir, "estgt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).treefile")
net_file = joinpath(checkpoint_dir, "estnets_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod).netfile")

if !isfile(estgt_file) touch(estgt_file) end

"/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/temp_data/estgt_n500_1_1000_500_low.treefile"

# Estimated gene trees

Estimated gene trees are generated as follows:

```bash
#!/bin/bash

cd $inphynet/simulation-study/from-sequences/scripts

for ntaxa in 500
do
    for ils in low med high
    do
        for ngt in 100 1000 5000
        do
            for seq_len in 500 1000 5000
            do
                for replicate in $(seq 1 5)
                do
                    for m in 25
                    do
                        julia -p3 --project=../../.. ./simulate_estimated_gts.jl ${ntaxa} ${replicate} ${ngt} ${seq_len} ${ils} ${m}
                    done
                done
            done
        done
    done
done
```

In [39]:
# Consolidate all estimated gene trees into a single file
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end

@everywhere GC.gc()
length(findall(i -> isassigned(est_gts, i), 1:ngt))

1000

# Subset decomposition

In [23]:
_, _, nj_tre = estimate_nj_tree(est_gts)
subsets = sateIdecomp(nj_tre, m)
1

[NJ] Calculating AGIC.
[NJ] Estimating NJ tree.


1

# Generate Condor data

In [32]:
df_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/CFs/"
nj_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/NJs/"
nruns = 10
GC.gc()

for (i, subset_taxa) in enumerate(subsets)
    output_file = "$(net_file)_$(i)"
    runtime_file = "$(output_file).runtime"
    output_net_file = "$(output_file).netfile"
    
    if isfile(output_net_file) && isfile(runtime_file)
        log("SNaQ $(i)", "Already inferred.")
        continue
    end

    temp_gts = Array{HybridNetwork}(undef, length(est_gts))
    for i = 1:length(est_gts)
        temp_gts[i] = pruneTruthFromDecomp(est_gts[i], subset_taxa)
    end

    # 1. Quartets
    q, t = countquartetsintrees(temp_gts)
    df = silently() do
        readTableCF(writeTableCF(q, t))
    end
    CSV.write(joinpath(df_dir, "df_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).csv"), writeTableCF(df))

    # 2. Starting trees
    init_tree = pruneTruthFromDecomp(nj_tre, subset_taxa)
    tre_out = joinpath(nj_dir, "nj_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).tre")
    open(tre_out, "w+") do f
        write(f, writeTopology(init_tree))
    end

    # 3. Write info to condor input table
    tab_file = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/condor/inputs.tab"
    open(tab_file, "a") do f
        write(f, "$(ntaxa),$(replicatenum),$(ngt),$(seq_len),$(ils_level),$(m),$(i)\n")
    end

    # 4. Constraints inferred in Condor
end
@everywhere GC.gc()

Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 12650 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Read

# Infer constraints

This is done in Condor with the submit file `simulation-study/condor/submit.submit`

# InPhyNet

In [24]:
# Load estimated data
# est_constraints = Array{HybridNetwork}(undef, length(subsets))
est_constraints = Vector{HybridNetwork}([])
for i = 1:length(subsets)
    net_file = joinpath(
        "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/snaq_nets/",
        "snaq_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).out"
    )
    if !isfile(net_file)
        continue
    end

    net = split(readlines(net_file)[1], " -Ploglik")[1]
    push!(est_constraints, readTopology(net))
    # est_constraints[i] = readTopology(net)
end
println("$(length(est_constraints))/$(length(subsets)) constraints")

# est_constraint_runtimes = Vector{Float64}(
#     [parse(Float64, readlines("$(net_file)_$(i).runtime")[1]) for i=1:length(subsets)]
# )

15/21 constraints


In [25]:
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end
est_D, est_namelist = calculateAGIC(est_gts)
est_namelist = Vector{String}(est_namelist)
D, namelist = majorinternodecount(true_net)
1

InterruptException: InterruptException:

In [26]:
for c in est_constraints
    rootatnode!(c, getchildren(c.node[c.root])[1])
end

In [27]:
[c.numTaxa for c in est_constraints]
[getNetDistances(c, pruneTruthFromDecomp(true_net, [leaf.name for leaf in c.leaf])) for c in est_constraints]

15-element Vector{Int64}:
 2
 0
 2
 2
 0
 0
 0
 2
 0
 0
 0
 8
 0
 0
 2

In [29]:
inphynet_time = @elapsed mnet = netnj(est_D, est_constraints, est_namelist, force_unrooted = true)

rootatnode!(mnet, "OUTGROUP")
rootatnode!(true_net, "OUTGROUP")
hardwiredClusterDistance(mnet, true_net, true), hardwiredClusterDistance(majorTree(mnet), majorTree(true_net), true)

#  1/21 constraints: (146, 78)
#  7/21 constraints: (192, 132)
# 10/21 constraints: (230, 170)
# 12/21 constraints: 
# not bad, but WORSE with more constraints??
# hopefully this will alleviate itself once we get all constraints

SolutionDNEError: No valid (i, j) pairs exist for the given distance matrix.

In [28]:
for j in 1:length(est_constraints)
    inphynet_time = @elapsed mnet = netnj(D, est_constraints[1:j], namelist, force_unrooted = true)

    rootatnode!(mnet, "OUTGROUP")
    rootatnode!(true_net, "OUTGROUP")
    println("$(j): $(hardwiredClusterDistance(mnet, true_net, true)), $(hardwiredClusterDistance(majorTree(mnet), majorTree(true_net), true))")
end

1: 754, 724
2: 730, 700
3: 718, 688
4: 694, 664
5: 662, 634
6: 626, 598
7: 612, 584


SolutionDNEError: No valid (i, j) pairs exist for the given distance matrix.

In [30]:
inphynet_time = @elapsed mnet = netnj(D, est_constraints, namelist, force_unrooted = true)

SolutionDNEError: No valid (i, j) pairs exist for the given distance matrix.

In [28]:
for e in true_net.edge e.length = -1. end
println(writeTopology(majorTree(true_net)))

(((((((t3,t4),(((((t22,t23),t18),t5),(((t13,(t17,(t19,t20))),t12),(((t8,t9),(t10,t11)),(t6,(t16,((t24,t25),t21)))))),((t2,(t7,(t14,t15))),t1))),(((((((t233,(t235,(t241,t242))),(t243,t244)),t229),(t227,(t228,(t230,t231)))),((t226,((t247,t248),(((t239,t238),((t249,t250),t234)),(t245,t246)))),(t236,t237))),(t232,t240)),(((t257,t258),(((t267,(t274,t275)),t261),t251)),((t253,(((t264,t265),t260),t254)),((((t266,(t272,t273)),t259),(t255,((t262,t263),t256))),(t252,((t268,t269),(t270,t271)))))))),((t76,(((((t92,t93),t85),((((t90,t91),t82),(t83,t84)),t95)),(t78,t79)),((((((t99,t100),t88),(t86,t87)),(t97,t98)),t77),(((t94,t96),t89),(t80,t81))))),(((((t160,t161),(t167,t168)),((t162,t163),(t151,t155))),((t158,t159),((((t153,t154),(t156,t157)),(t165,t166)),(((((t174,t175),((t170,t171),t169)),t164),(t172,t173)),t152)))),(((((((t195,t196),(t191,t192)),(t180,t181)),((((t187,t188),t184),t183),t176)),(t177,(((t199,t200),(t185,(t189,t190))),((t179,t186),((t197,t198),t182))))),(t178,(t193,t194))),((((t401,

# Save results

In [ ]:
save_estimated_gts_results(
    "$(ntaxa)", true_net, replicatenum, ngt, ils_level,
    m, dmethod, seq_len, mnet, est_constraints,
    est_gts, est_constraint_runtimes, inphynet_time
)